# Example: Using the Time to First Detection (TTFD) Plot Type To Assess Plume Migration and Monitoring Network Performance in an NRAP-Open-IAM Control File Simulation.

This example demonstrates how to use the TTFD plot type in the Control File Interface. The figures and .csv files produced show (1) how dissolved CO$_2$ plumes (areas with a relative change in dissolved CO$_2$ concentration > 20 %) spread through two aquifers in a simulation with 30 stochastic realizations, (2) the times at which the monitoring wells provided (through x, y, and z coordinates) could detect these plumes, and (3) the probability that a dissolved CO$_2$ plume forms in different regions of the aquifers.

The TTFD plot type is run after the simulation has been completed. In order to use the plot type, one must set up the system model to include certain components and certain component outputs. For example, the system model must have an aquifer component that produces plume size metrics corresponding with the type of plume being assessed (e.g., Dissolved_CO2_dx, Dissolved_CO2_dy, Dissolved_CO2_dz outputs for TTFD plots examining dissolved CO$_2$ plumes in the aquifer).

Note that before this example can be run, you need to have the OpenIAMEnv environment activated (or whatever you named your environment containing the required python libraries - see the installation .txt files in the installers folder). For example, you can open Anaconda Prompt, use the command "conda activate OpenIAMEnv" to activate the environment, navigate to the directory for NRAP-Open-IAM, and then use the command "jupyter notebook" to open Jupyter Notebook from that directory and within that environment. Then, the jupyter notebook examples you run will have access to your NRAP-Open-IAM environment.

This example uses a library called yamlmagic to read the input shown further below (i.e., the code beneath the statement '%load_ext yamlmagic'). To install yamlmagic, open a command prompt (e.g., Anaconda Prompt), activate your NRAP-Open-IAM environment, type 'pip install yamlmagic', and hit enter. After yamlmagic is installed and your NRAP-Open-IAM environment is activated, then use the instructions above to open Jupyter Notebook.

First, run the lines below to load the yaml and yamlmagic libraries.

In [ ]:
# First, load yaml and yamlmagic
import yaml

%load_ext yamlmagic

The material below is the Control File input. We discuss each section further below ('ModelParams', 'Stratigraphy', and 'LookupTableReservoir1', 'MultisegmentedWellbore1', 'FutureGen2AZMI1', 'FutureGen2AZMI2', and 'Plots').

In [ ]:
%%yaml cfi_yaml_data

#-------------------------------------------------
ModelParams:
    EndTime: 15
    TimeStep: 1
    Analysis:
        type: lhs
        siz: 30
    Components: [LookupTableReservoir1, MultisegmentedWellbore1,
                 FutureGen2AZMI1, FutureGen2AZMI2]
    OutputDirectory: output/JupyterNotebook_TTFD_CFI_example
    Logging: Info
#-------------------------------------------------
Stratigraphy:
    numberOfShaleLayers: 5
    shale1Thickness: 198.7
    shale2Thickness: 74.4
    shale3Thickness: 110.3
    shale4Thickness: 118.9
    shale5Thickness: 530.4
    aquifer1Thickness: 33.2
    aquifer2Thickness: 84.1
    aquifer3Thickness: 31.1
    aquifer4Thickness: 61.6
    reservoirThickness: 7
#-------------------------------------------------
LookupTableReservoir1:
    Type: LookupTableReservoir
    FileDirectory: source/components/reservoir/lookuptables/FutureGen2/1008_sims
    TimeFile: time_points.csv
    ParameterFilename: parameters_and_filenames_trunc.csv
    Parameters:
        index: 5
    Outputs: [pressure,
              CO2saturation]
#-------------------------------------------------
MultisegmentedWellbore1:
    Type: MultisegmentedWellbore
    Connection: LookupTableReservoir1
    Number: 2
    Locations:
        coordx: [236377.0, 236758.0]
        coordy: [4409269.5, 4410107.6]
    Parameters:
        wellRadius: 0.05715
        logAqu1Perm:
            value: -13.39
            vary: False
        logAqu2Perm:
            value: -11.05
            vary: False
        logAqu3Perm:
            value: -12.48
            vary: False
        logAqu4Perm:
            value: -11.92
            vary: False
        logWellPerm:          # range allowed: -101 to -9
            min: -14
            max: -11
            value: -13
        brineDensity: 1030.9
        CO2Density: 775.0
        brineViscosity: 7.5e-4
        CO2Viscosity: 6.6e-5
    Outputs: [CO2_aquifer1, CO2_aquifer2, CO2_aquifer3, CO2_aquifer4, CO2_atm,
              brine_aquifer1, brine_aquifer2, brine_aquifer3, brine_aquifer4, brine_atm]
#-------------------------------------------------
FutureGen2AZMI1:
    Type: FutureGen2AZMI
    Connection: MultisegmentedWellbore1
    AquiferName: aquifer1
    Parameters:
        # aqu_thick (33.2 m for aquifer 1) and depth (1044 m) parameter do not need
        # to be set up as they are linked to stratigraphy
        por: 0.118
        log_permh: -13.39
        log_aniso: 0.3
        rel_vol_frac_calcite: 0.1
    Outputs: [pH_volume, TDS_volume, Dissolved_CO2_volume,
              Dissolved_CO2_dx, Dissolved_CO2_dy, Dissolved_CO2_dz]
#-------------------------------------------------
FutureGen2AZMI2:
    Type: FutureGen2AZMI
    Connection: MultisegmentedWellbore1
    AquiferName: aquifer3
    Parameters:
        # aqu_thick (31.1 m for aquifer 3) and depth (742 m) parameter do not need
        # to be set up as they are linked to stratigraphy
        por: 0.132
        log_permh: -12.48
        log_aniso: 0.3
        rel_vol_frac_calcite: 0.1
    Outputs: [pH_volume, TDS_volume, Dissolved_CO2_volume,
              Dissolved_CO2_dx, Dissolved_CO2_dy, Dissolved_CO2_dz]
#-------------------------------------------------
Plots:
    pH_plume_volumes:
        TimeSeriesAndStats: [pH_volume]
        subplot:
            use: True
            ncols: 2
    TDS_plume_volumes:
        TimeSeriesAndStats: [TDS_volume]
        subplot:
            use: True
            ncols: 2
    Dissolved_CO2_volumes:
        TimeSeries: [Dissolved_CO2_volume]
        subplot:
            use: True
            ncols: 2
    TTFD_Plot:
        TTFD:
            PlumeType: Dissolved_CO2
            FigureSize: (15, 12)
            FigureDPI: 300
            PlotInjectionSites: True
            InjectionCoordx: 2.37e5
            InjectionCoordy: 4.41e6
            MonitoringLocations:
                coordx: [236377.0, 236758.0, 236377.0, 236758.0]
                coordy: [4409269.5, 4410107.6, 4409269.5, 4410107.6]
                coordz: [-1015, -715, -1015, -715] # Near the tops of aquifers 1 and 3
                HorizontalWindow: 1
                VerticalWindow: 5
            ComponentNameList: [FutureGen2AZMI1, FutureGen2AZMI2]
#-------------------------------------------------

First, we review the different sections of the .yaml control file. Control files will typically use the pound symbol followed by hyphens (#---) to mark separate sections, but these characters are not required. Any symbols following a pound symbol (#) will be ignored by the Python script that reads the .yaml file, so the hyphens are only there as a visual aid. Every control file must have a section called 'ModelParams'. In this section, one specifies the model times to assess ('EndTime: 15' and 'TimeStep: 1', times are in years), the type of analysis to conduct (i.e., deterministic or stochastic, controlled by the 'Analysis' entry), the names of the components included in the system model ('Components'), and the output directory to save the output ('OutputDirectory'). Here, the analysis type is lhs ('type: lhs'), which stands for Latin Hypercube Sampling. This lhs simulation will use 30 stochastic realizations ('siz: 30'). Each entry is followed by a colon (:), with the input following the colon. Some entries take a single input value (e.g., 'EndTime: 15.') or a list of input values ('Components: [LookupTableReservoir1, MultisegmentedWellbore1, FutureGen2AZMI1, FutureGen2AZMI2]'). Some entries have other entries indented beneath them; the indented entry is on a lower line and preceeded by additional spaces (e.g., 'type' and 'siz' are indented beneath 'Analysis').

In [ ]:
 # These statements show what was entered for the ModelParams section:
print('ModelParams:')

print(yaml.dump(cfi_yaml_data['ModelParams']))

Next, the control file specifies the stratigraphy of the model domain. All unit thicknesses are in meters. This stratigraphy is based on the FutureGen 2.0 site. Higher unit numbers are closer to the surface and shales and aquifers always alternate. For example, shale 1 ('shale1Thickness') is just above the reservoir ('reservoirThickness'), aquifer 1 ('aquifer1Thickness') is just above shale 1, and shale 2 ('shale2Thickness') is just above aquifer 1. Spatially variable stratigraphy can be used in control files (see control file examples 33a to 38).

In [ ]:
# These statements show what was entered for the Stratigraphy section:
print('Stratigraphy:')

print(yaml.dump(cfi_yaml_data['Stratigraphy']))

Here, a LookupTableReservoir component is used to portray reservoir conditions over time. This component reads reservoir pressures and CO$_2$ saturations from .csv files included in the FutureGen 2.0 data set (https://edx.netl.doe.gov/dataset/futuregen-2-0-1008-simulation-reservoir-lookup-table). Before this example can be run, the .zip file available on that web page must be downloaded. Then, the contents of the .zip file must be extracted and placed in the folder /source/components/reservoir/lookuptables/FutureGen2/1008_sims (where the source folder is within the NRAP-Open-IAM directory).

The name of the reservoir component is 'LookupTableReservoir1'; the component name is dictated by the first line in the component's section. The component type is specified through the entry 'Type: LookupTableReservoir', and the component outputs are specified through the entry 'Outputs: [pressure, CO2saturation]'. The details of the .csv files containing the output of reservoir simulations are presented through the entries 'FileDirectory', 'TimeFile', 'ParameterFilename', and 'Parameters: index: 5'. All of the .csv files used are in the directory entered for 'FileDirectory'. The index provided ('index: 5') specifies which row to use in the file entered under 'ParameterFilename' (different rows in that file contain different .csv file names, which each correspond with a particular reservoir simulation). The .csv file provided for the 'TimeFile' entry contains the times used in the reservoir simulations.

In [ ]:
# These statements show what was entered for the LookupTableReservoir1 section:
print('LookupTableReservoir1:')

print(yaml.dump(cfi_yaml_data['LookupTableReservoir1']))

The pressures and CO$_2$ saturations from the LookupTableReservoir component are then provided to a MultisegmentedWellbore component called 'MultisegmentedWellbore1'. The connection between the LookupTableReservoir1 component and this wellbore component is specified through the entry 'Connection: LookupTableReservoir1'. A MultisegmentedWellbore component will be made for two locations, as specified through the entry 'Number: 2'. These locations are specified through the coordx and coordy entries (easting and northing distances, in meters). The parameter values entered under 'Parameters' specify factors like the wellbore radius ('wellRadius', in meters), the permeability of the wellbore along shale units ('logWellPerm'), and the wellbore permability across differrent aquifers (e.g., 'logAqu1Perm' for aquifer 1 and 'logAqu2Perm' for aquifer 2). Permeabilities are given in log$_{10}$ m$^2$. The logWellPerm parameter will vary stochastically between the minimum and maximum values provided ('min' and 'max'). The component outputs are specified under 'Outputs'. Note that MultisegmentedWellbore components have other parameters that are not included here (e.g., compressibility). Any parameters that are not provided values will automatically be assigned default parameter values.

In [ ]:
# These statements show what was entered for the MultisegmentedWellbore1 section:
print('MultisegmentedWellbore1:')

print(yaml.dump(cfi_yaml_data['MultisegmentedWellbore1']))

The CO$_2$ and brine leakage rates produced by the MultisegmentedWellbore component are given as input to two FutureGen2AZMI aquifer components. Each aquifer component calculates the sizes of contaminant plumes over time. One component ('FutureGen2AZMI1') is set up to represent aquifer 1, while the other ('FutureGen2AZMI2') is set up to represent aquifer 3. This assignment is performed through the 'AquiferName' entry. While the MultisegmentedWellbore component was connected to the LookupTableReservoir component, each aquifer component is connected to the wellbore component through the entry 'Connection: MultisegmentedWellbore1'. The thickness and depth of the corresponding aaquifer will automatically be assigned to each component, as is specified by the silenced text within the 'Parameters' subsection (i.e., the text preceeded by '#'). The Parameters section also specifies the porosity ('por'), permeability ('log_permh'), anisotropy ratio ('log_aniso'), and calcite within the unit ('rel_vol_frac_calcite'). The 'Outputs' entry specifies all of the output metrics to be produced by the aquifer component. The use of the TTFD plot for dissolved CO$_2$ plumes (as shown here) requires the inclusion of the Dissolved_CO2_dx, Dissolved_CO2_dy, and Dissolved_CO2_dz metrics.

In [ ]:
# These statements show what was entered for the FutureGen2AZMI1 section:
print('FutureGen2AZMI1:')

print(yaml.dump(cfi_yaml_data['FutureGen2AZMI1']))

# These statements show what was entered for the FutureGen2AZMI2 section:
print('FutureGen2AZMI2:')

print(yaml.dump(cfi_yaml_data['FutureGen2AZMI2']))

Next, the .yaml control file specifies the figures to be produced through the 'Plots' section. Three of the plots ('pH_plume_volumes', 'TDS_plume_volumes', and 'Dissolved_CO2_volumes') show different metrics over time. For more information regarding these plot types ('TimeSeriesAndStats' and 'TimeSeries'), see section 2.7 of the NRAP-Open-IAM User Guide ('Setup of visualization options').

The name of the TTFD plot entry is 'TTFD_Plot', and it is specified as a TTFD plot through the inclusion of 'TTFD' indented beneath the plot name. All of the options for the TTFD plot type are then indented under 'TTFD'. Two of these entries are required: 'PlumeType' and 'ComponentNameList'. 'PlumeType' specifies the kind of plumes being examined, while 'ComponentNameList' specifies the names of the aquifer components that produce the required output. The 'PlotInjectionSites' option specifies whether to show the injection site in the figures produced, and the 'InjectionCoordx' and 'InjectionCoordy' entries specify the injection site's location. The 'FigureDPI' entry specifies the dots-per-inch (dpi) of the figures. The options indented under the 'MonitoringLocations' entry specify details regarding the monitoring equipment used to detect contaminant plumes. The x, y, and z values (easting, northing, and depths, all in meters) are entered as lists following the 'coordx', 'coordy', and 'coordz' entries. For the 'coordz' entry, depths beneath the surface are entered as negative values. The 'HorizontalWindow' and 'VerticalWindow' entries specify the horizontal and vertical distances (m) within which the sensor will detect plumes. The 'VerticalWindow' input can be regarded as the vertical extent of sensors along the monitoring well, with sensors extending above and below each 'coordz' value.

In [ ]:
# These statements show what was entered for the Plots section:
print('Plots:')

print(yaml.dump(cfi_yaml_data['Plots']))

When the control file is run, the TTFD plot entry will produce three types of map-view figures: (1) figures showing the timing of plume development in the aquifer(s) considered, (2) figures showing the eariest times at which the monitoring locations provided could detect the plumes (i.e., when the location is within a plume), and (3) figures showing the probability that a plume forms in different parts of the aquifer(s) considered. The first two types of figures are made for each of the 30 realizations in the simulation, while only one probability figure is created. The probability is calculated as the number of realizaitons in which a plume occured at a particular location divided by the total number of realizations. All results will also be saved in .csv files. Running an NRAP-Open-IAM control file depends on the type of computer used and how it was installed; for more information, see section 1.3 of the User Guide ('Installing NRAP-Open-IAM').

A control file is normally run through a command prompt (e.g., Anaconda Prompt on Anaconda Navigator). Below, we use the input above to write a .yaml control file. Then, we run the control file through a script-based approach.

Note that this simulation takes some time to run (e.g., about two to three minutes). The output will be saved to a folder called 'JupyterNotebook_TTFD_CFI_example' in the output folder of your NRAP-Open-IAM directory. The control file interface will print messages regarding the status of the simulation (e.g., 'Running file' or 'Analysis completed'). Once the simulation has finished and the TTFD plots are being made, you can watch the figures appear in the 'JupyterNotebook_TTFD_CFI_example' folder. Once the TTFD analysis has finished, the code below will print the statement 'Finished'.

In [ ]:
 import sys
import os

# Add the source folder to the path
sys.path.insert(0, os.sep.join(['..', '..', 'source']))

source_dir = os.path.join(os.getcwd(), '..', '..', 'source')

# Save yaml data into file to be run
example_directory = os.path.join(os.getcwd(), '..', 'Control_Files', 'examples_for_jupyter_notebook')

output_directory = os.path.join(os.getcwd(), '..', '..', 'output')

example_output_directory = os.path.join(output_directory, 'JupyterNotebook_TTFD_CFI_example')

file_name = 'ControlFile_JupyterNotebook_TTFD_example.yaml'

os.chdir(example_directory)

# Write the .yaml file so it can be read by openiam_cf.py
with open(file_name, 'w') as f:
    yaml.dump(cfi_yaml_data, f)

# Path to the file openiam_cf.py, which is used to run control files
run_file = os.path.join(source_dir, 'openiam', 'openiam_cf.py')

# Set up the command to run the control file
run_command = '"{0}" --file "{1}"'.format(
     run_file, file_name)

print('Running Control File example...')

# Run the control file
%run -i $run_command

print('Finished.')

Now, wait for the control file to finish running. The TTFD plot type will continue to run for some time after the code above prints '100% 30 of 30 samples collected' (indicating that all 30 of the realizations are complete). When the TTFD plots are finished, the code above will print the statement 'Finished'.

It will make figures for each of the 30 realizations. You can check the output folder ('JupyterNotebook_TTFD_CFI_example') to see when figures are produced. If the figures have been produced, the code below loads three of the figures. Otherwise, it will print a statement saying that the figure could not be found.

The figure 'Dissolved_CO2_Plume_Timings_Realization_0.png' shows the spatiotemporal evolution of dissolved CO$_2$ plumes in the first realization (index 0). There are four subplots, and each subplot shows one quarter of the depth range used in the simulation (which is set by the stratigraphy). The top of the figure has the statement 'Layers with Lower Times Shown Above Other Layers' because each subplot contains different layers for different depth intervals. Depth intervals that never had any plumes will not have any color-labelled results shown. Because these figures are meant to focus on the earliest times at which plumes occur, layers that have later times are shown beneath other layers. The results for all depth intervals are saved in .csv files, however. The code below also loads this figure type for the second and third realizations (indices 1 and 2).

The figure 'Dissolved_CO2_Monitoring_TTFD_Realization_0.png' shows when the dissolved CO$_2$ plumes first arrive at the monitoring wells specified in the first realization (index 0). The code below also loads this figure type for the second and third realizations (indices 1 and 2).

The figure 'Dissolved_CO2_Plume_Probabilities.png' shows the probability that a dissolved CO$_2$ plume will occur in different regions of the two aquifers considered (aquifer 1 and aquifer 3). Like the first figure, this probability figure has four subplots that each shows results for one quarter of the depth range and each subplot contains multiple depth intervals.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

figure_names = ['Dissolved_CO2_volumes.png', 'Dissolved_CO2_Plume_Timings_Realization_0.png',
                'Dissolved_CO2_Monitoring_TTFD_Realization_0.png', 'Dissolved_CO2_Plume_Timings_Realization_1.png',
                'Dissolved_CO2_Monitoring_TTFD_Realization_1.png', 'Dissolved_CO2_Plume_Timings_Realization_2.png',
                'Dissolved_CO2_Monitoring_TTFD_Realization_2.png',
                'Dissolved_CO2_Plume_Probabilities.png']

for figRef in range(len(figure_names)):
    file_path = os.path.join(example_output_directory, figure_names[figRef])
    
    check_for_figure = os.path.exists(file_path)
    
    if check_for_figure:
        img = mpimg.imread(file_path)
        
        # fig, ax = subplots(figsize=(20, 16))
        fig = plt.figure(figRef + 1, figsize=(15, 12), dpi=300)
        ax = plt.gca()
        
        ax.imshow(img, interpolation='nearest')
        
        # We do not need the ticks from imshow, the figure file has its own ticks
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        
        # We also do not need the spline added by imshow
        plt.setp(ax.spines.values(), alpha = 0)
    else:
        print('The figure {} was not found.'.format(file_path))

plt.show()

Because the 'logWellPerm' parameter is set to vary between minimum and maximum values, each realization has a different 'logWellPerm' value. These differences will cause different leakage rates, which are then provided to the aquifer components ('FutureGen2AZMI1' and 'FutureGen2AZMI2'). The plume volumes produced by the aquifer components will vary with the leakage rates provided by the wellbore component ('MultisegmentedWellbore1').

In the time series plot ('Dissolved_CO2_volumes.png'), locations 0 and 1 are the first and second locations entered for MultisegmentedWellbore1 ('coordx: [first_x, second_x]', 'coordy: [first_y, second_y]', given in meters). If you change these locations or add more locations, the TTFD plots will change. For example, making the well locations be closer to the injection well location (x = 2.37e5 m, y = 4.41e6 m) will generally cause higher leakage rates and higher plume volumes.

The TTFD plot type can be used to examine worst-case scenarios of wellbore leakage and demonstrate that certain monitoring well locations can quickly detect contaminant plumes in an aquifer.

The TTFD plot type can also produce the .iam files used as input to the DREAM tool (Design for Risk Evaluation And Management), which was also developed by NRAP. Such files will be produced if the entry for the TTFD plot contains the option 'WriteDreamOutput: True' indented under 'TTFD' (see control file example 41). DREAM can be used to optimize a monitoring network given simulated contaminant plumes in an aquifer.